In [1]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

## Import JSON 

In [2]:
zip_dir  = "../data/zip2"

files = glob(os.path.join(zip_dir, '*'))
for filename in files:
    if '.zip' in filename:
        pass
    elif '.json' in filename:
        pass
    else:
        newfile = filename+'.json'
        os.rename(filename, newfile)

jsons = glob(os.path.join(zip_dir, '*.json'))

In [3]:
# jsons = jsons[-2:]
jsons

['../data/zip2/part-v003-o001-r-00000.json',
 '../data/zip2/part-v003-o001-r-00001.json',
 '../data/zip2/part-v003-o001-r-00002.json',
 '../data/zip2/part-v003-o001-r-00003.json',
 '../data/zip2/part-v003-o001-r-00004.json',
 '../data/zip2/part-v003-o001-r-00005.json',
 '../data/zip2/part-v003-o001-r-00006.json',
 '../data/zip2/part-v003-o001-r-00007.json',
 '../data/zip2/part-v003-o001-r-00008.json',
 '../data/zip2/part-v003-o001-r-00009.json']

In [ ]:
# jsons = ['../data/part_v003_o001_r_00000.json', ..., '../data/part_v003_o001_r_00001.json']

dfrts = []
dfors = []
for i, json in enumerate(jsons):
    # Instance Preparation
    dates = []
    tweets = []
    
    start = time.time()
    # Load json file: date and tweet
    with open(json, 'r') as file:
        pet_parse = ijson.parse(file, multiple_values=True)
        for prefix, event, value  in pet_parse:
            if prefix == 'created_at':
                dates.append(datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y'))
            if prefix == 'text':
                tweets.append(value)
    print ("Loading json elapsed_time:{0}".format(time.time() - start) + "[sec]")
    
    # Df
    data = np.vstack([dates, tweets]).T

    df = pd.DataFrame(data, columns=['date', 'tweet'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    df['date'] = df['date'].dt.date
    df['RT_flag'] = df['tweet'].str.contains('RT') # True if tweet containt 'RT', otherwise False

    # divide df by Original tweet and RT 
    dfrt = df[df['RT_flag'] == True]
    dfor = df[df['RT_flag'] == False]
    
    # Save to csv file
    dfor.to_csv('../data/dfs2/dfor'+str(i)+'.csv')
    dfrt.to_csv('../data/dfs2/dfrt'+str(i)+'.csv')
    
    # Append to dfs
    dfrts.append(dfor)
    dfors.append(dfor)

Loading json elapsed_time:1150.1346881389618[sec]
Loading json elapsed_time:974.1615660190582[sec]
Loading json elapsed_time:710.9687736034393[sec]
Loading json elapsed_time:726.949878692627[sec]


## Read json dfs from csv

In [2]:
zip_dir  = "../data/dfs2"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    df = df.dropna(subset=['tweet'])
    dfors.append(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## df info

In [6]:
c = [len(df) for df in dfors]
print('The amount of original tweets(without falts):', sum(c))
alltwt = sum(c)

The amount of original tweets(without falts): 8464087


In [37]:
# c = [len(df) for df in dfors]
# alltwt = sum(c)
# goodtwt = 8464087

# print('The amount of original tweets(include falts):', alltwt)
# print('# falts:', alltwt - goodtwt)
# print('portion of good twt:', goodtwt/alltwt)

The amount of original tweets(include falts): 8699476
# falts: 235389
portion of good twt: 0.9729421634130607


In [ ]:
c = [len(df) for df in dfors]
alltwt = sum(c)
goodtwt = 8464087

print('The amount of original tweets(include falts):', alltwt)
print('# falts:', alltwt - goodtwt)
print('portion of good twt:', goodtwt/alltwt)

## Create csv by day

In [40]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_day(dfors, filetype="T"):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    
    # tweets volume
    vol = []
    
    # Find tweets
    for dfor in tqdm(dfors):
        
        vol.append(len(dfor)) # tweets volume
        
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True)
        
        dfs.append(dft)

    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    print('Original tweets volume:', sum(vol))
    
    # Save
    outname = filetype+'_original2.csv'
    outdir = '../results/rtjp/day'

    if not os.path.exists(outdir):
            os.makedirs(outdir)

    savename = os.path.join(outdir, outname)    

    dfnew.to_csv(savename)

    return dfnew

# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# dfnew = findtwt_day(dfors, filetype="C")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [04:34<00:00, 27.50s/it]


Original tweets volume: 8464087


## Run day

In [41]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# for filetype in filetypes:
#     dfnew = findtwt_day(dfors, filetype=filetype)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,714,514,195,167,298,242,247,130,153,31,...,72,77,175,78,530,18,53,32,55,94
2020-02-01,2727,2386,854,706,1319,1402,952,596,705,223,...,265,565,658,352,473,85,151,176,261,331
2020-02-02,2663,2673,851,785,1379,1569,910,568,610,196,...,314,705,773,392,180,91,148,163,296,378
2020-02-03,2579,2142,820,713,1653,1543,856,603,685,177,...,217,816,638,343,141,74,173,179,249,362
2020-02-04,2708,1967,847,682,1666,1509,859,606,680,192,...,254,864,676,343,155,70,143,163,291,353
2020-02-05,2622,2006,766,702,2252,1531,864,617,657,218,...,250,801,634,319,156,66,256,165,264,338
2020-02-06,2576,1917,761,621,1676,1385,777,601,656,162,...,257,712,612,329,137,57,155,175,257,353
2020-02-07,2582,1980,721,676,1464,1355,826,620,691,148,...,237,725,647,334,150,61,114,167,268,358
2020-02-08,2630,2469,769,754,1533,1412,899,595,644,211,...,271,546,679,412,163,66,172,165,268,408


In [42]:
# # Save
# outname = 'c_original2.csv'
# outdir = '../results/jp'

# if not os.path.exists(outdir):
#         os.makedirs(outdir)
    
# savename = os.path.join(outdir, outname)    

# dfnew.to_csv(savename)

## Create csv by hour

In [3]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfors, filetype="T"):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 

    # ************ここから勝負***************************
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        dfs.append(dfor_c)
        
    # Connect dfs 
    print('concat dfs')
    for i, df in enumerate(dfs):
        if i == 0:
            dfall = df
        else:
            dfall = pd.concat([dfall, df], axis=0)
    print('Connected dfs')
    
    # Make rows
    uni_dates = dfall['date'].tolist()
    uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
    
    df_date = dfall.groupby('date') # df grouped by date   
        
    for date in uni_dates:
        df_oneday = df_date.get_group(date) # df of one day
        dfh = df_oneday.groupby('hour').sum()
        dfh = dfh.drop(['year', 'month'],axis=1)

        # Save to csv
        outname = filetype+str(date)+'.csv'
        outdir = '../results/rtjp/hour'

        if not os.path.exists(outdir):
                os.makedirs(outdir)

        savename = os.path.join(outdir, outname)
        dfh.to_csv(savename)

# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# findtwt_hour(dfors, filetype="T")


# findtwt_hour(dfors, filetype="T")
          

In [4]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [06:33<00:00, 39.39s/it]


concat dfs
Connected dfs


100%|██████████| 10/10 [01:49<00:00, 10.93s/it]


concat dfs
Connected dfs


100%|██████████| 10/10 [02:21<00:00, 14.12s/it]


concat dfs
Connected dfs


100%|██████████| 10/10 [04:07<00:00, 24.80s/it]


concat dfs
Connected dfs


## Check data created  

In [19]:
dft = pd.read_csv('../results/jp/hour/T2020-02-20.csv')
dft.T1.sum()

,hour,T1,T2,T3,T4,T5,T6,T7,T8,T9,...,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21
0,0.0,252,180,92,186,257,97,188,135,113,...,6,5,10,4,13,1,4,6,11,4
1,1.0,242,174,117,185,261,92,199,104,107,...,5,5,11,3,15,6,5,5,10,7
2,2.0,295,195,91,168,329,107,227,119,104,...,4,3,20,7,19,2,3,3,5,5
3,3.0,488,257,154,274,600,167,374,209,238,...,11,7,20,4,28,1,5,8,13,10
4,4.0,322,187,124,241,396,133,273,128,148,...,3,6,15,9,12,2,1,9,12,2
5,5.0,284,159,124,191,318,105,243,130,124,...,6,4,16,6,17,1,1,7,6,4
6,6.0,285,165,143,207,346,104,229,123,136,...,10,11,6,5,18,2,4,5,5,6
7,7.0,306,148,166,236,412,128,248,126,133,...,15,3,25,7,16,3,4,4,14,17
8,8.0,394,210,162,295,442,148,296,194,194,...,3,8,11,4,22,3,6,6,16,7
9,9.0,419,247,219,320,594,162,400,219,251,...,12,12,22,9,29,5,1,7,20,8
